# Automating feature transformations with SageMaker Data Wrangler, Pipelines, and Feature Store

This notebook shows you how to create a SageMaker Pipeline along with an AWS Lambda function to automate feature transformations and ingestion into Feature Store, triggered off of new data files that are uploaded to S3. It assumes that you already have already created a Data Wrangler `.flow` file, which is done within module 05 (`05-module-scalable-batch-ingestion`) of this training workshop.

The notebook has three main sections:
1.  General setup
2.	Creating a SageMaker Pipeline which:
    - Performs the transformations contained in a Data Wrangler `.flow` file stored in Amazon S3 using a SageMaker Processing Job 
    - Stores the transformed features in the Amazon SageMaker Feature Store
3.	Creating an AWS Lambda function which:
    - Is triggered whenever any new data is uploaded to S3
    - Updates the `.flow` file to reference the new dataset
    - Triggers the SageMaker Pipeline with the new `.flow` file
    
**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.t3.medium`

First, we need to copy these variables from the Data Wrangler generated notebook from the previous step: 

In [ ]:
# SageMaker Python SDK version 2.x is required
import sagemaker
import subprocess
import sys
import os
import uuid
import json
import time
import boto3
from zipfile import ZipFile
import inspect

#module containing utility functions for this notebook
import pipeline_utils

original_version = sagemaker.__version__
if sagemaker.__version__ < "2.48.1":
    subprocess.check_call(
        [sys.executable, "-m", "pip", "install", "sagemaker==2.48.1"]
    )
    import importlib
    importlib.reload(sagemaker)
    
# S3 bucket for saving processing job outputs
# Feel free to specify a different bucket here if you wish.
sess = sagemaker.Session()
bucket = sess.default_bucket()
sm_client = boto3.client('sagemaker')
iam_role = sagemaker.get_execution_role()
region = sess.boto_region_name

In [ ]:
print('SageMaker version: ' + sagemaker.__version__)
print(f'Default bucket: {bucket}')
print(f'IAM Role: {iam_role}')

### Let's define bunch of useful prefix vars for Feature Store and Data Wrangler

In [ ]:
# Location of data files 
workshop_prefix = 'fscw'
s3_feature_store_workshop_prefix = f'sagemaker-feature-store/{workshop_prefix}'

s3_data_wrangler_flow_files_prefix = f'{s3_feature_store_workshop_prefix}/data_wrangler_flows'
data_wrangler_flow_file = 'DWF-Orders.flow'

# Flow URI is taken from DataWrangler Flow UI previously created
s3_data_wrangler_flow_uri=f's3://{bucket}/{s3_data_wrangler_flow_files_prefix}/{data_wrangler_flow_file}'
print(f's3_data_wrangler_flow_uri: {s3_data_wrangler_flow_uri}')

s3_data_landing_zone_prefix = f'{s3_feature_store_workshop_prefix}/data_landing_zone'
print(f's3_data_landing_zone_prefix: {s3_data_landing_zone_prefix}')

s3_feature_store_data_files_prefix = f'{s3_feature_store_workshop_prefix}/data'
fs_orders_file = 'orders.csv'

s3_feature_store_orders_file = f'{s3_feature_store_data_files_prefix}/{fs_orders_file}'
print(f's3_feature_store_orders_file: {s3_feature_store_orders_file}')

### NOTE: The "%store -r" command reads variable values set by the previous notebook outputs

This method ensures that we get the correct current value for critical variables like the Feature Group name.

In [ ]:
# Read the name of the Feature Group created in previous notebook
# For example: 'fscw-orders-07-06-19-19'

%store -r feature_group_name
print(feature_group_name)

# DataWrangler Flow file output name is taken from the auto-generated value in previous notebook

%store -r output_name
print(output_name)

### Locate Feature Groups created in previous notebooks

Several feature groups were created by the Jupyter notebook in module-5 
notebook

In [ ]:
# Now let's lookup feature group names that start with our special prefix
filter_subword = workshop_prefix
fg_summaries = sm_client.list_feature_groups(NameContains=filter_subword)['FeatureGroupSummaries']

for fg in fg_summaries:
    print(fg['FeatureGroupName']) 

### Let's lookup the schema for the FeatureGroups we want to load.

We can extract this information from the Feature Store via the `describe_feature_group` method which returns a list of `FeatureDefinitions` that contains the column name and data type. In our case, we want to load the `orders` feature group which we reference by name.

In [ ]:
order_feature_defs = sm_client.describe_feature_group(
    FeatureGroupName=feature_group_name)['FeatureDefinitions']

order_feature_defs

In [ ]:
# Collect list of column names from Feature Definition
order_column_names = []
for feature_def in order_feature_defs:
    order_column_names.append(feature_def['FeatureName'])

order_column_names

In [ ]:
query_columns = ", ".join(order_column_names)
print(query_columns)

## Query Historical Records in Feature Store
First, let's ensure that the records we processed from the original processing job made it into the Feature Store successfully. Note that once the job is complete, it can still take several minutes for all records to be replicated from the online store to the offline store.

In [ ]:
def get_resolved_s3uri_base(fg_name):
    fg_config = sm_client.describe_feature_group(FeatureGroupName=fg_name)
    s3_uri_full = fg_config['OfflineStoreConfig']['S3StorageConfig']['ResolvedOutputS3Uri']
    s3_uri_base, last = os.path.split(s3_uri_full)
    print (f'Found Resolved S3 Uri Base: {s3_uri_base}')
    return s3_uri_base

orders_fg_s3_path = get_resolved_s3uri_base(feature_group_name)
print(f'Orders Feature Group S3 path to Offline data: \n {orders_fg_s3_path}')

In [ ]:
df = pipeline_utils.get_offline_store_data(feature_group_name, 
                s3_uri=orders_fg_s3_path, column_list=query_columns)

In [ ]:
print(df.shape)
df.dtypes

## Update policy of SageMaker Studio execution role 
As part of automation in this notebook, you will create IAM roles to assign to AWS Lambda. To do that, you first need to give some permission to am IAM execution role. You can provide those permissions by adding the following as an [inline policy](https://docs.aws.amazon.com/IAM/latest/UserGuide/access_policies_manage-edit.html#edit-inline-policy-console).

If you are running this notebook in Amazon SageMaker Studio, the IAM role assumed by your Studio user needs permission to create AWS Lambda functions and IAM roles. To provide this permission to the role, do the following.

1. Open the [Amazon SageMaker console](https://console.aws.amazon.com/sagemaker/).
2. Select Amazon SageMaker Studio and choose your user name.
3. Under **User summary**, copy just the name part of the execution role ARN 
5. Go to the [IAM console](https://console.aws.amazon.com/iam) and click on **Roles**. 
6. Find the role associated with your SageMaker Studio user
7. Under the Permissions tab, click **Add inline policy** and enter the following in the JSON tab:
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "IAMPolicy1",
            "Effect": "Allow",
            "Action": [
                "iam:CreatePolicy",
                "iam:AttachRolePolicy",
                "iam:CreateRole"
            ],
            "Resource": [
                "*"
            ]
        },
        {
            "Sid": "IAMPolicy2",
            "Effect": "Allow",
            "Action": [
                "iam:PassRole"
            ],
            "Resource": [
                "*"
            ],
            "Condition": {
                "StringEquals": {
                    "iam:PassedToService": [
                        "lambda.amazonaws.com"
                    ]
                }
            }
        },
        {
            "Sid": "LambdaFunction",
            "Effect": "Allow",
            "Action": [
                "lambda:CreateFunction",
                "lambda:UpdateFunctionCode",
                "lambda:AddPermission"
            ],
            "Resource": "*"
        },
        {
            "Sid": "S3Notification",
            "Effect": "Allow",
            "Action": [
                "s3:PutBucketNotification"
            ],
            "Resource": "*"
        },
        {
            "Sid": "STSPermission",
            "Effect": "Allow",
            "Action": [
                "sts:GetCallerIdentity"
            ],
            "Resource": "*"
        }
    ]
}
```
 

## Create a SageMaker Pipeline from the Data Wrangler Flow
The transformations we defined in Data Wrangler are encapsulated in a `.flow` file. We will parameterize our SageMaker pipeline with the S3 URI of a new input flow file we will create on the fly once new data is made available in S3. 

In [ ]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.4xlarge"
)

# Note: this parameter will be replaced by the Lambda function
input_flow= ParameterString(
    name='InputFlow',
    default_value='s3://placeholder-bucket/placeholder.flow'
)

# Output configuration if any passed as processing job container arguments 
output_config = {}

In [ ]:
from sagemaker.processing import Processor

container_id = pipeline_utils.get_container(region)

container_uri=f"{container_id}.dkr.ecr.{region}.amazonaws.com/sagemaker-data-wrangler-container:1.x"
print(container_uri)

In [ ]:
# Create Processor instance

processor = Processor(
    role=iam_role,
    image_uri=container_uri,
    instance_count=processing_instance_count,
    instance_type=processing_instance_type
)

In [ ]:
from sagemaker.processing import FeatureStoreOutput
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep

step_process = ProcessingStep(
    name="fscw-Automated-DataWrangler-Processing",
    processor=processor,
    inputs=[
        ProcessingInput(input_name='flow', 
                        destination='/opt/ml/processing/flow',
                        source=input_flow,
                        s3_data_type= 'S3Prefix',
                        s3_input_mode= 'File'
                       )
    ],
    outputs=[
        ProcessingOutput(
            output_name=output_name,
            app_managed=True, 
            feature_store_output=FeatureStoreOutput(feature_group_name=feature_group_name))
    ],
    job_arguments=[f"--output-config '{json.dumps(output_config)}'"]
)

In [ ]:
from sagemaker.workflow.pipeline import Pipeline

pipeline_basename = "featurestore-ingest-pipeline"

pipeline_name=f"{pipeline_basename}-{time.strftime('%d-%H-%M-%S', time.gmtime())}"

print(pipeline_name)

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        input_flow
    ],
    steps=[step_process],
    sagemaker_session=sess
)

In [ ]:
pipeline.upsert(iam_role)

Now we have a Pipeline set up ready to execute when called with a new input flow file. Now we'll create a lambda function that will automatically create a new flow file when new data is uploaded to S3. 

## Creating a Lambda Function triggered off of S3
We have a provided a file `pipeline_utils.py` which contains some helper functions we can use to create a lambda function containing our custom code. 

### Setup IAM Roles
AWS Lambda needs permissions to be able to call other AWS services. These permissions are provided by IAM roles. We first create the IAM role that will be assumed by AWS Lambda and then assign permissions to it.

We now set variables that will be used to setup the automation. The default placeholder values will work but you can update them as well, if you wish.

In [ ]:
# Create IAM role for Lambda function

role_name = f"sm-lambda-role-{time.strftime('%d-%H-%M-%S', time.gmtime())}"
fcn_name = f"sm-lambda-fcn-{time.strftime('%d-%H-%M-%S', time.gmtime())}"

account_num = boto3.client('sts').get_caller_identity()['Account']

#Create IAM role for the Lambda function
lambda_role = pipeline_utils.create_role(role_name)

#Wait for the role to be activated
print('Waiting for 30 seconds for the newly created role to be active.')
time.sleep(30)
print('30 seconds are up; proceeding with rest of the execution.')

Once the Lambda function is created, we zip it into a deployment package ready for upload onto AWS Lambda. Once the package is ready, we create the AWS Lambda function using the IAM role created earlier.

In [ ]:
#Create code for AWS Lambda function
print(f'Data Wrangler Flow Uri passed to Lambda: {s3_data_wrangler_flow_uri}')

In [ ]:
lambda_code = pipeline_utils.create_lambda_fcn(s3_data_wrangler_flow_uri, pipeline_name)

In [ ]:
print(lambda_code)

In [ ]:
#Zip AWS Lambda function code
#Write code to a .py file
with open('lambda_function.py', 'w') as f:
    f.write(inspect.cleandoc(lambda_code))
#Compress file into a zip
with ZipFile('function.zip','w') as z:
    z.write('lambda_function.py')
#Use zipped code as AWS Lambda function code
with open('lambda_function.py', 'w') as f:
    f.write(lambda_code)

#Create AWS Lambda function
with open('function.zip', 'rb') as f:
    fcn_code = f.read()   

lambda_arn = pipeline_utils.create_lambda(fcn_name, fcn_code, lambda_role['arn'])

Lastly, we setup Amazon S3 to trigger AWS Lambda whenever a new CSV file is uploaded into the S3 bucket under the `prefix` specified earlier. 

In [ ]:
# Create and attach trigger for Amazon S3 event to kick-off AWS Lambda function
print(f'Data landing zone prefix for S3 trigger: {s3_data_landing_zone_prefix}')
pipeline_utils.create_s3_trigger(fcn_name, bucket, s3_data_landing_zone_prefix, account_num, lambda_arn)

#Wait for the trigger to be created
print('Waiting for 5 seconds for the newly created trigger to be active.')
time.sleep(5)

We have now successfully setup up all the necessary pieces of infrastructure. Now we will try and test the setup by uploading a CSV file into your Amazon S3 Bucket and monitor the pipeline execution. 

## Validate Feature Pipeline

Now that you have configured the automated pipeline, you can test that everything has been setup correctly. To do that, we will leverage a small CSV file that contains orders data under our partitions directory (`../data/partitions/2021-5/partition.csv`). In the cell below, we will upload this file to a designated location in our Amazon S3 bucket. This will trigger our Lambda function which will kick-off the automated pipeline that you have just setup.

In [ ]:
# This file contains a small sample of orders data
sagemaker.s3.S3Uploader.upload("../05-module-scalable-batch-ingestion/orders.csv", 
                               f"s3://{bucket}/{s3_data_landing_zone_prefix}")
#wait for file to finish uploading 
time.sleep(5)

### S3 file upload will trigger SageMaker Pipelines execution

The uploading of a file to the designated S3 location above will trigger a Lambda function to execute. This Lambda function will then execute the reference Pipeline. You can use SageMaker Studio to navigate to the Pipeline which should appear similar to the image below.

<img src="images/automated-datawrangler-pipeline.png" />

In [ ]:
# check pipeline execution 
summaries = sm_client.list_pipeline_executions(PipelineName=pipeline_name).get('PipelineExecutionSummaries')
summaries

In [ ]:
latest_execution = sm_client.list_pipeline_executions(PipelineName=pipeline_name).get('PipelineExecutionSummaries')[0].get('PipelineExecutionArn')
print (latest_execution)

In [ ]:
# Wait for pipeline execution to complete 'Executing' status

response = sm_client.describe_pipeline_execution(PipelineExecutionArn=latest_execution)
print('Pipeline Execution status: ' + response['PipelineExecutionStatus'])

while response['PipelineExecutionStatus'] == 'Executing':
    print('Pipeline is still in Executing status...')
    time.sleep(60)
    response = sm_client.describe_pipeline_execution(PipelineExecutionArn=latest_execution)
    print('Pipeline Execution status: ' + response['PipelineExecutionStatus'])


print('Pipeline is done Executing')

You can monitor the pipeline run inside the Pipelines section of Studio. Once the execution completes, we can test retrieval of a record from the Feature Store. This will confirm that the Data Wrangler job executed by the Pipeline ran successfully.

In [ ]:
# Lookup valid Record ID for get_record call
record_id = 'O2235'

feature_store_runtime = sess.boto_session.client('sagemaker-featurestore-runtime', region_name=region)
sample_record = feature_store_runtime.get_record(FeatureGroupName=feature_group_name, 
                                                 RecordIdentifierValueAsString=str(record_id))

In [ ]:
sample_record

### Clean-up
To avoid recurring charges you need to: 
* Delete the datasets uploaded to S3.
* Stop any running Data Wrangler and Jupyter Notebook instances within Studio when not in use.
* Delete the feature group.
* Delete the SageMaker Pipelines pipeline
* Delete the AWS Lambda function that was created
* Remove the Event notification that was setup on the Amazon S3 bucket to trigger the pipeline on upload of new file
